**IMPORTANTE:** 
- Il runtime di default è **CPU** per non consumare le ore sulle **GPU**, ricordarsi di cambiarlo;

**Collegamento a myDrive per download del dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Clone del repository github**

In [ ]:
!git clone https://github.com/fgiacome/MLDL23-FL-project.git

In [ ]:
import sys
sys.path.append('./MLDL23-FL-project/')
%cd MLDL23-FL-project

**Training and test function**

In [ ]:
# Useful function for model training and testing
# Import Intersect Over Union
from utils.stream_metrics import StreamSegMetrics
from utils.utils import MeanReduction

# Training function
def train(net, data_loader, loss_function, optimizer):

  # Loop initialization
  cumulative_loss = 0
  samples = 0
  mean_iou = StreamSegMetrics(n_classes = 16, name = 'Mean IoU')
  reduction = MeanReduction()

  # Set the training mode to the model
  net.train()

  # Loop over batches
  for idx, (X, y) in enumerate(data_loader):

    # Send data to GPU
    X = X.cuda()
    # y = y.long()
    y = y.cuda()

    # Predictions
    y_hat = net(X)['out']
    y_pred = torch.argmax(y_hat, dim=1)

    # Compute loss
    criterion = loss_function(y_hat, y)
    loss = reduction(criterion, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Update training variables
    cumulative_loss += loss.item() 
    samples += X.size(0)
    mean_iou.update(y.cpu().numpy(), y_pred.cpu().numpy())

  return cumulative_loss / samples, mean_iou.get_results()



# Test function
def test(net, data_loader, loss_function):

  # Test variables initialization
  cumulative_loss = 0
  samples = 0
  mean_iou = StreamSegMetrics(n_classes = 16, name = 'Mean IoU')
  reduction = MeanReduction()

  # Set the evaluation mode to the model
  net.eval()

  # Test loop (we don't want to compute the gradients)
  with torch.no_grad():

    # Loop over batches
    for idx, (X, y) in enumerate(data_loader):

      # Send data to the GPU
      X = X.cuda()
      # y = y.long()
      y = y.cuda()

      # Compute predictions
      y_hat = net(X)['out']
      y_pred = torch.argmax(y_hat, dim = 1)
      
      # Loss computation
      criterion = loss_function(y_hat, y)
      loss = reduction(criterion, y)

      # Update test variables
      cumulative_loss += loss.item()
      samples += X.size(0)
      mean_iou.update(y.cpu().numpy(), y_pred.cpu().numpy())

  return cumulative_loss / samples, mean_iou.get_results()

**Hyperparams to test**

In [ ]:
# Hyperparameters
# values: 
# - beta \in {1e-4, 1e-3, 1e-2} (see how it affects on paper [12] on FDA)
# - lr \in {1e-3, 1e-2, 1e-1} (We use Adam since we have seen that it provides 
#                              better results without a optimal fine-tuning)
# - transforms \in {True, False}
# || TOTAL COMBINATIONS = 18 ||
# Parameters lists
beta_list = [2e-3, 4e-3, 6e-2]
lr_list = [1e-3, 1e-2, 1e-1]
transforms_list = [True, False]

# If we have to do FDA, please set FDA = True
FDA = False

# Folder name 
foldername = 'CP_lr1e'

# Combinations initialization
if FDA:
  idx = 0
  hyperparameters = dict()
  for transforms in transforms_list:
    for beta in beta_list:
      for lr in lr_list:
        hyperparameters[f'comb{idx + 1}'] = dict()
        hyperparameters[f'comb{idx + 1}']['transforms'] = transforms
        hyperparameters[f'comb{idx + 1}']['beta'] = beta
        hyperparameters[f'comb{idx + 1}']['lr'] = lr
        idx += 1
else: 
  idx = 0
  hyperparameters = dict()
  for transforms in transforms_list:
    for lr in lr_list:
      hyperparameters[f'comb{idx + 1}'] = dict()
      hyperparameters[f'comb{idx + 1}']['transforms'] = transforms
      hyperparameters[f'comb{idx + 1}']['lr'] = lr
      idx += 1

# Results path
results_path = '/content/drive/MyDrive/mldl-project-2b/FDA_P1_Models_Checkpoints/'

**CONTROLLA LA COMBINAZIONE LE FACCIAMO TUTTE CON TRANSFORMS = TRUE**

In [ ]:
hyperparameters['comb6']

**ALE HO VISTO CHE NON HAI CONTROLLATO, CONTROLLA PER FAVORE**

**NO FDA TESTS**

In [ ]:
# Importations
import main
import torch
import torch.nn as  nn
import random
from utils.utils import MeanReduction
from datasets.idda import IDDADataset
from datasets.GTA import GTADataset
from models import deeplabv3, mobilenetv2
################################################################################
############################## CHOOSE COMB! ####################################
################################################################################
comb = 'comb6' #################################################################
################################################################################
############################## CHOOSE COMB! ####################################
################################################################################
# Fix random seed
random.seed(300890)

# args for dataset and model importation
class args:
  dataset = 'idda'
  model = 'deeplabv3_mobilenetv2'

# Model
model = main.model_init(args).cuda()

# Transforms
if hyperparameters[comb]['transforms'] == True:
  gta_transforms, idda_transforms = main.get_transforms(args)
else: 
  _, gta_transforms = main.get_transforms(args)
  _, idda_transforms = main.get_transforms(args)

# Importations
import os
import json

# Initialization of train IDDA dataset (centralized)
root = 'data/idda'
client_id = "CENTRALIZED"
with open(os.path.join(root, 'train.json'), 'r') as f:
    all_data = json.load(f)
idda_img_names = [] 
for k in all_data.keys():
    for i in all_data[k]:
      idda_img_names.append(i)

# Generate train-validation sets
idda_df = IDDADataset(root = root, list_samples = idda_img_names, 
                            transform = idda_transforms, client_name = client_id)
  
# Initialization of test IDDA dataset
with open(os.path.join(root, 'test_same_dom.txt'), 'r') as f:
    test_same_dom_data = f.read().splitlines()
    test_same_dom_df = IDDADataset(root = root, list_samples = test_same_dom_data, 
                                        transform = idda_transforms,
                                        client_name='test_same_dom')

with open(os.path.join(root, 'test_diff_dom.txt'), 'r') as f:
    test_diff_dom_data = f.read().splitlines()
    test_diff_dom_df = IDDADataset(root = root, list_samples = test_diff_dom_data, 
                                        transform = idda_transforms,
                                        client_name='test_diff_dom')

# GTA5 directory
root = '/content/drive/MyDrive/MLDL_Datasets/GTA5/'
filename = 'train.txt'
full_path = root + filename

# Import GTA5 images
lines_clean = []
with open(full_path, 'r') as fp:
  lines = fp.readlines()
for i, line in enumerate(lines):
  if line[-1] == '\n': 
    lines[i] = line[:-1]
  if os.path.exists(root + '/images/' + lines[i]):
    lines_clean.append(lines[i])

# GTA5 initialization
gta_df = GTADataset(root = root, list_samples = lines_clean,
                    transform = gta_transforms, client_name='GTA5')

# Data Loaders
gta_loader = torch.utils.data.DataLoader(gta_df, 
                                         batch_size = 8,
                                         shuffle = True, num_workers = 2)
idda_loader = torch.utils.data.DataLoader(idda_df, batch_size = 16,
                                          shuffle = True, num_workers = 2)
test_same_dom_loader = torch.utils.data.DataLoader(test_same_dom_df, 
                                                   batch_size = 16, 
                                                   shuffle = True, num_workers=2)
test_diff_dom_loader = torch.utils.data.DataLoader(test_diff_dom_df, 
                                                   batch_size = 16,
                                                   shuffle=True, num_workers=2)

# Criterion
criterion = nn.CrossEntropyLoss(ignore_index = 255, reduction = 'none')

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), 
                             lr = hyperparameters[comb]['lr'])

# Number of epochs and checkpoint
num_epochs = 20
checkpoint = 5 # The model is saved every 5 epochs 

# Initializaion of loss and miou lists for each dataset
gta_loss_list = []
idda_loss_list = []
test_same_dom_loss_list = []
test_diff_dom_loss_list = []
gta_miou_list = []
idda_miou_list = []
test_same_dom_miou_list = []
test_diff_dom_miou_list = []

# Results path
foldername += str(len(str(hyperparameters[comb]['lr'])) - 2) + '_' 
foldername += str(hyperparameters[comb]['transforms'])[0]

# Loop on epochs
for epoch in range(num_epochs):

  # Train the model
  gta_loss, gta_miou = train(model, gta_loader, criterion, optimizer)

  # Append results
  gta_loss_list.append(gta_loss)
  gta_miou_list.append(gta_miou)

  # Model evaluation
  if (epoch + 1) % checkpoint == 0:

    checkpoint_path = results_path + foldername + f'/checkpoint_e{epoch + 1}.json'
    
    # IDDA dataset
    idda_loss, idda_miou = test(model, idda_loader, criterion)

    # Test datasets
    test_same_dom_loss, test_same_dom_miou = test(model, test_same_dom_loader, criterion)
    test_diff_dom_loss, test_diff_dom_miou = test(model, test_diff_dom_loader, criterion)

    # Append results 
    idda_loss_list.append(idda_loss)
    idda_miou_list.append(idda_miou)
    test_same_dom_loss_list.append(test_same_dom_loss)
    test_same_dom_miou_list.append(test_same_dom_miou)    
    test_diff_dom_loss_list.append(test_diff_dom_loss)
    test_diff_dom_miou_list.append(test_diff_dom_miou)    

    # Save model 
    torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'results': {
                      'GTA': {
                          'Loss': gta_loss,
                          'Mean IoU': gta_miou
                      },
                      'IDDA': {
                          'Loss': idda_loss,
                          'Mean IoU': idda_miou
                      },
                      'Test Same Dom': {
                          'Loss': test_same_dom_loss,
                          'Mean IoU': test_same_dom_miou
                      },
                      'Test Diff Dom': {
                          'Loss': test_diff_dom_loss,
                          'Mean IoU': test_diff_dom_miou
                      }
                  },
                }, checkpoint_path)
    
    print('-----------------------------------------------------')
    print(f'\tEpoch: {epoch + 1}')
    print('\t GTA loss {:.5f}, GTA Mean IoU {:.2f}'.format(gta_loss,
          gta_miou['Mean IoU']))
    print('\t IDDA loss {:.5f}, IDDA Mean IoU {:.2f}'.format(idda_loss,
          idda_miou['Mean IoU']))
    print('\t Test same dom loss {:.5f}, Test same dom Mean IoU {:.2f}'.format(test_same_dom_loss,
          test_same_dom_miou['Mean IoU']))
    print('\t Test diff dom loss {:.5f}, Test diff dom Mean IoU {:.2f}'.format(test_diff_dom_loss,
          test_diff_dom_miou['Mean IoU']))
    print('-----------------------------------------------------')

**Save results**

In [ ]:
# Import json
import json

# Path
results_path = '/content/drive/MyDrive/mldl-project-2b/FDA_P1_Models_Checkpoints/Results_default/'
results_path += foldername
results_path += '.json'

# Save results on a dict
results_dict = {
                'GTA': (gta_loss_list, gta_miou_list), 
                'IDDA': (idda_loss_list, idda_miou_list),
                'Test Same Dom': (test_same_dom_loss_list, 
                                    test_same_dom_miou_list),
                'Test Diff Dom': (test_diff_dom_loss_list, 
                                    test_diff_dom_miou_list)
              }

# Save data on file system (Remember to download it!)
with open(results_path, 'w') as fp:  
  json.dump(results_dict, fp)

- RandomFDA, inizializzare solo con beta;
- servono add_style e get_style_from_img 

chiamata statica
RandomFourierDomainAdaptation.get_style_from_img,

- l'immagine da passare deve essere PIL (su importazione)
- l'immagine da chiamare è dal target dataset (adatto gta sul client)
devo ciclare idda dataset e per tutte le immagine devo chiamare get_style_from_img (apri manualmente con open).

- salvare np.ndarray in una variabile provvisoria